# Estudo de oportunidades para PBZPA

In [1]:
# helper function
from os import listdir
def dir_files(name: str, folder: str = None):
    if folder:
        files = [folder + e for e in listdir(folder) if (e[-4:] == ".xls") or (e[-4:] == ".csv")]
    else:
        files = [e for e in listdir() if (e[-4:] == ".xls") or (e[-4:] == ".csv")]
    print(f"{name}: {', '.join(files)}")
    return files

In [57]:
from pandas import read_excel
from pandas import read_csv

public_private = dir_files('Aeródromos públicos e privados', r'E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/')
aerodromos_publicos =  read_excel(public_private[1], sheet_name=0, header=[1,2])
aerodromos_privados = read_excel(public_private[0], sheet_name=0, header=[0,1])

print('\n')

aisweb = dir_files('AISWeb', r'E:/Users/falam/IdeaProjects/Aerodromos/AISWeb/')
aisweb =  read_csv(aisweb[0], sep=',', encoding='latin-1', dtype={'TEMP':object, 'AIP':object, 'NOTAM':object})

aisweb.insert(loc=4, column="Categoria",value='')
icao_aeropublicos = aerodromos_publicos.iloc[:,0].values
icao_aeroprivados = aerodromos_privados.iloc[:,0].values
for i, icao in enumerate(aisweb.ICAO):
    if icao in icao_aeropublicos:
        aisweb.at[i, 'Categoria'] = 'Público'
    elif icao in icao_aeroprivados:
        aisweb.at[i, 'Categoria'] = 'Privado'

Aeródromos públicos e privados: E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/Aeródromos Privados_28MAR2019.xls, E:/Users/falam/IdeaProjects/Aerodromos/ANAC/Beta/public_private/Aeródromos Públicos_28MAR2019.xls


AISWeb: E:/Users/falam/IdeaProjects/Aerodromos/AISWeb/output_aisweb_c788f86d66.csv


In [217]:
# ajuste temporário até proximo scrapping - já foi ajustado no projeto.
from numpy import nan
for i, status in enumerate(aisweb.Status):
    if aisweb.Status[i] == 'OK':
        aisweb.at[i, 'Status'] = nan

In [220]:
aisweb.head()

,timestamp,ICAO,CIAD,Aeródromo,Categoria,Cidade,UF,Status,TEMP,AIP,NOTAM,Debug
0,2019-04-23T05:39:18.613346-03:00,SSOK,PR0088,14 Bis,Privado,LONDRINA,PR,,[],[],"[['E0479/19', 'R', 'E0032/19', '13/03/2019 23:...",{'response_status_code': 200}
1,2019-04-23T05:39:23.555346-03:00,SIJQ,MT0099,A2 Aviação Agrícola Ltda,Privado,ALTA FLORESTA,MT,,[],[],[],{'response_status_code': 200}
2,2019-04-23T05:39:28.227306-03:00,SNDH,BA0132,ABA,Privado,BARREIRAS,BA,,[],[],[],{'response_status_code': 200}
3,2019-04-23T05:39:33.417345-03:00,SIGN,GO0048,ACGC,Privado,MIMOSO DE GOIÁS,GO,,[],[],[],{'response_status_code': 200}
4,2019-04-23T05:39:37.673311-03:00,SDGA,MS0029,Adecoagro,Privado,ANGÉLICA,MS,,[],[],[],{'response_status_code': 200}


In [155]:
aisweb.count()

timestamp    2630
ICAO         2630
CIAD         2569
Aeródromo    2568
Categoria    2630
Cidade       2568
UF           2568
Status       2630
TEMP         2569
AIP          2569
NOTAM        2569
Debug        2630
dtype: int64

### Há valores nulos?

In [219]:
aisweb.isnull().sum()

timestamp     0
ICAO          0
CIAD         61
Aeródromo    62
Categoria     0
Cidade       62
UF           62
Status        0
TEMP         61
AIP          61
NOTAM        61
Debug         0
dtype: int64

### Aeródromos não encontrados (fechado, localidade cancelada, Localidade reservada aguardando portaria da ANAC)

#### Aeródromos que constam na lista da ANAC, mas não foram encontrados no AISWeb

In [115]:
aero_naoencontrado = aisweb[aisweb.Status == "O aeródromo não foi encontrado"]
print(f"Total de aeródromos ({aero_naoencontrado.Status[:1].item()}): {aero_naoencontrado.ICAO.count()}\n")
aero_naoencontrado

Total de aeródromos (O aeródromo não foi encontrado): 3


#### Localidade cancelada

In [150]:
aero_localizacao_cancelada = aisweb[aisweb.Status == "Localidade cancelada"]
print(f"Total de aeródromos ({aero_localizacao_cancelada.Status[:1].item()}): {aero_localizacao_cancelada.ICAO.count()}\n")
aero_localizacao_cancelada

Total de aeródromos (Localidade cancelada): 3



,timestamp,ICAO,CIAD,Aeródromo,Categoria,Cidade,UF,Status,TEMP,AIP,NOTAM,Debug
572,2019-04-23T05:41:27.979460-03:00,SSXI,NaN,NaN,Privado,NaN,NaN,Localidade cancelada,NaN,NaN,NaN,{'response_status_code': 200}
916,2019-04-23T05:47:54.303460-03:00,SITM,NaN,NaN,Privado,NaN,NaN,Localidade cancelada,NaN,NaN,NaN,{'response_status_code': 200}
1563,2019-04-23T05:51:38.710223-03:00,SJHV,NaN,NaN,Privado,NaN,NaN,Localidade cancelada,NaN,NaN,NaN,{'response_status_code': 200}


#### Aeródromos com localidade reservada aguardando portaria da ANAC

In [166]:
aero_localizacao_reservada = aisweb[aisweb.Status == "Localidade reservada. Aguardando portaria da ANAC."]
print(f"Total de aeródromos ({aero_localizacao_reservada.Status[:1].item()}): {aero_localizacao_reservada.ICAO.count()}\n")
aero_localizacao_reservada

Total de aeródromos (Localidade reservada. Aguardando portaria da ANAC.): 54



,timestamp,ICAO,CIAD,Aeródromo,Categoria,Cidade,UF,Status,TEMP,AIP,NOTAM,Debug
1160,2019-04-23T05:45:07.399269-03:00,SJZR,NaN,NaN,Privado,NaN,NaN,Localidade reservada. Aguardando portaria da A...,NaN,NaN,NaN,{'response_status_code': 200}
1166,2019-04-23T05:45:31.220450-03:00,SDYG,NaN,NaN,Privado,NaN,NaN,Localidade reservada. Aguardando portaria da A...,NaN,NaN,NaN,{'response_status_code': 200}
2081,2019-04-23T05:58:18.273535-03:00,SSCV,NaN,NaN,Público,NaN,NaN,Localidade reservada. Aguardando portaria da A...,NaN,NaN,NaN,{'response_status_code': 200}
2098,2019-04-23T05:54:56.418299-03:00,SSGG,NaN,NaN,Público,NaN,NaN,Localidade reservada. Aguardando portaria da A...,NaN,NaN,NaN,{'response_status_code': 200}
2206,2019-04-23T05:54:47.379295-03:00,SDKE,NaN,NaN,Público,NaN,NaN,Localidade reservada. Aguardando portaria da A...,NaN,NaN,NaN,{'response_status_code': 200}
2212,2019-04-23T05:55:11.708334-03:00,SDLI,NaN,NaN,Público,NaN,NaN,Localidade reservada. Aguardando portaria da A...,NaN,NaN,NaN,{'response_status_code': 200}
2247,2019-04-23T05:58:12.845497-03:00,SDXB,NaN,NaN,Público,NaN,NaN,Localidade reservada. Aguardando portaria da A...,NaN,NaN,NaN,{'response_status_code': 200}
2250,2019-04-23T05:58:21.539945-03:00,SDYW,NaN,NaN,Público,NaN,NaN,Localidade reservada. Aguardando portaria da A...,NaN,NaN,NaN,{'response_status_code': 200}
2253,2019-04-23T05:58:30.384906-03:00,SIBW,NaN,NaN,Público,NaN,NaN,Localidade reservada. Aguardando portaria da A...,NaN,NaN,NaN,{'response_status_code': 200}
2256,2019-04-23T05:54:28.362335-03:00,SICA,NaN,NaN,Público,NaN,NaN,Localidade reservada. Aguardando portaria da A...,NaN,NaN,NaN,{'response_status_code': 200}


#### Aeródromos fechados até determinada data

In [8]:
filtro = "Aeródromo FECHADO até"
aero_fechado = aisweb[aisweb.Status.str.contains(filtro) == True]
print(f"Total de aeródromos fechados: {aero_fechado.ICAO.count()}")

Total de aeródromos fechados: 165


# TEMP

#### Aeródromos contendo a string "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO"

In [209]:
filtro = "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO"
ad_clsd_pbzpa = aisweb[aisweb.TEMP.str.contains(filtro) == True]
print(f"Total de aeródromos com \"AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO\": {ad_clsd_pbzpa.ICAO.count()}\n")

Total de aeródromos com "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO": 182



#### Fechamento não revogado

In [175]:
ad_clsd_pbzpa_nao_rev = ad_clsd_pbzpa[ad_clsd_pbzpa.Status.str.contains('Aeródromo FECHADO', regex=True)]
print(f"Total de prospects para novos negócios: {ad_clsd_pbzpa_nao_rev.ICAO.count()}\n")

Total de prospects para novos negócios: 157



#### Fechamento Agendado ou AD CLSD Revogado

In [216]:
ad_clsd_pbzpa_rev = ad_clsd_pbzpa[~ad_clsd_pbzpa.Status.str.contains('Aeródromo FECHADO')]
print(f"Total de aeródromos com \"AD CLSD\" revogado: {ad_clsd_pbzpa_rev.ICAO.count()}\n")
ad_clsd_pbzpa_rev

Total de aeródromos com "AD CLSD" revogado: 25



,timestamp,ICAO,CIAD,Aeródromo,Categoria,Cidade,UF,Status,TEMP,AIP,NOTAM,Debug
23,2019-04-23T05:41:39.082458-03:00,SDIO,SP0109,Aeroclube de Itápolis,Privado,ITÁPOLIS,SP,OK,"[['Revogado', 'INFOTEMP 1090/2018', 'AD CLSD D...",[],"[['F0671/19', 'R', 'F0175/19', '12/04/2019 14:...",{'response_status_code': 200}
63,2019-04-23T05:39:59.456312-03:00,SJQL,MG0130,Águas Claras Aviação Agrícola,Privado,SANTA JULIANA,MG,OK,"[['Revogado', 'INFOTEMP 0015/2019', 'AD CLSD D...",[],[],{'response_status_code': 200}
84,2019-04-23T05:42:17.088425-03:00,SJYI,RR0092,Arai,Privado,PACARAIMA,RR,OK,"[['INFOTEMP 1208/2018', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
144,2019-04-23T05:42:43.850458-03:00,SIKK,MG0102,Calciolândia,Privado,ARCOS,MG,OK,"[['INFOTEMP 0216/2019', 'AD CLSD DEVIDO NAO AP...",[],"[['F0642/19', 'R', 'F0159/19', '10/04/2019 20:...",{'response_status_code': 200}
240,2019-04-23T05:41:14.508458-03:00,SSWB,DF0008,COOPADF,Privado,BRASÍLIA,DF,OK,"[['Revogado', 'INFOTEMP 0027/2019', 'AD CLSD D...",[],[],{'response_status_code': 200}
270,2019-04-23T05:44:16.382486-03:00,SSWH,SP0261,Dr. Emílio Fehr,Privado,SÃO CARLOS,SP,OK,"[['INFOTEMP 0140/2019', 'AD INSERIR NO RMK: DE...",[],[],{'response_status_code': 200}
347,2019-04-23T05:40:53.134471-03:00,SIPG,MT0115,Fazenda Água Preta,Privado,COCALINHO,MT,OK,"[['Revogado', 'INFOTEMP 0022/2019', 'AD CLSD D...",[],[],{'response_status_code': 200}
535,2019-04-23T05:43:20.385420-03:00,SWXX,MG0209,Fazenda Canadá,Privado,UBERLÂNDIA,MG,OK,"[['INFOTEMP 0217/2019', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
560,2019-04-23T05:40:13.167339-03:00,SJSN,GO0084,Fazenda Carvalho,Privado,SÃO LUIZ DO NORTE,GO,OK,"[['INFOTEMP 0219/2019', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
561,2019-04-23T05:40:18.605401-03:00,SDTO,SP0147,Fazenda Cataco,Privado,UBARANA,SP,OK,"[['Revogado', 'INFOTEMP 1070/2018', 'AD CLSD D...",[],[],{'response_status_code': 200}


#### Fechamento revogado, mas somente em missões de caráter emergencial
ou seja, candidatos para INFRACEA | Social. Da retorno de mídia. Será que há empresas com interesse na facilitação de acesso à região? Será há benefício fiscal para empresas que financiarem o produto PBZPA da INFRACEA nestas regiões?

In [162]:
filtro = "EXC ACFT OPS EM CARATER EMERGENCIAL"
ad_clsd_pbzpa_rev_emg = ad_clsd_pbzpa_rev[ad_clsd_pbzpa_rev.TEMP.str.contains(filtro, regex=True)]
print(f"Total de aeródromos com \"AD CLSD\" revogado em caráter emergencial: {ad_clsd_pbzpa_rev_emg.ICAO.count()}\n")
ad_clsd_pbzpa_rev_emg

Total de aeródromos com "AD CLSD" revogado em caráter emergencial: 33



,timestamp,ICAO,CIAD,Aeródromo,Categoria,Cidade,UF,Status,TEMP,AIP,NOTAM,Debug
76,2019-04-23T05:41:28.853458-03:00,SWAO,AM0043,Anamoim,Privado,SÃO GABRIEL DA CACHOEIRA,AM,Aeródromo FECHADO até 31/12/20: INFOTEMP 0163/...,"[['INFOTEMP 0163/2019', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
77,2019-04-23T05:41:36.210419-03:00,SDJS,RR0003,Andorinha,Privado,UIRAMUTÃ,RR,Aeródromo FECHADO até 31/12/20: INFOTEMP 1481/...,"[['Revogado', 'INFOTEMP 1159/2018', 'AD CLSD D...",[],[],{'response_status_code': 200}
98,2019-04-23T05:43:34.179424-03:00,SWAK,AM0042,Assunção do Içana,Privado,SÃO GABRIEL DA CACHOEIRA,AM,Aeródromo FECHADO até 31/12/20: INFOTEMP 0164/...,"[['INFOTEMP 0164/2019', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
115,2019-04-23T05:39:42.144311-03:00,SJYQ,RR0099,Barreirinha,Privado,UIRAMUTÃ,RR,Aeródromo FECHADO até 31/12/20: INFOTEMP 1153/...,"[['INFOTEMP 1153/2018', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
142,2019-04-23T05:42:31.950462-03:00,SJYR,RR0100,Cajú,Privado,UIRAMUTÃ,RR,Aeródromo FECHADO até 31/12/20: INFOTEMP 1152/...,"[['INFOTEMP 1152/2018', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
145,2019-04-23T05:42:49.878458-03:00,SJYS,RR0101,Camará,Privado,NORMANDIA,RR,Aeródromo FECHADO até 31/12/20: INFOTEMP 1147/...,"[['INFOTEMP 1147/2018', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
250,2019-04-23T05:42:21.223459-03:00,SWKU,AM0048,Cucuí,Privado,SÃO GABRIEL DA CACHOEIRA,AM,Aeródromo FECHADO até 31/12/20: INFOTEMP 0169/...,"[['INFOTEMP 0169/2019', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
1634,2019-04-23T05:53:12.297252-03:00,SDLM,RR0007,Guariba,Privado,NORMANDIA,RR,Aeródromo FECHADO até 31/12/20: INFOTEMP 1236/...,"[['INFOTEMP 1236/2018', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
1669,2019-04-23T05:51:25.680227-03:00,SJLJ,RR0051,Jatapuzinho,Privado,CAROEBE,RR,Aeródromo FECHADO até 31/12/20: INFOTEMP 1150/...,"[['INFOTEMP 1150/2018', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}
1696,2019-04-23T05:53:43.799262-03:00,SDMM,RR0009,Lago Verde,Privado,PACARAIMA,RR,Aeródromo FECHADO até 31/12/20: INFOTEMP 1238/...,"[['INFOTEMP 1238/2018', 'AD CLSD DEVIDO NAO AP...",[],[],{'response_status_code': 200}


# NOTAM

#### Aeródromos contendo a string "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO"

In [149]:
filtro = "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO"
ad_clsd_pbzpa = aisweb[aisweb.NOTAM.str.contains(filtro) == True]
print(f"Total de aeródromos com \"AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO\": {ad_clsd_pbzpa.ICAO.count()}\n")
ad_clsd_pbzpa

Total de aeródromos com "AD CLSD DEVIDO NAO APRESENTACAO DO PLANO BASICO DE ZONA DE PROTECAO": 11



,timestamp,ICAO,CIAD,Aeródromo,Categoria,Cidade,UF,Status,TEMP,AIP,NOTAM,Debug
2277,2019-04-23T05:56:09.774335-03:00,SNAM,MG0011,Santo Antônio do Amparo,Público,SANTO ANTÔNIO DO AMPARO,MG,OK,[],[],"[['F0660/19', 'N', '11/04/2019 18:38', 'AD CLS...",{'response_status_code': 200}
2361,2019-04-23T05:58:19.035908-03:00,SNMN,MG0075,Minas Novas,Público,MINAS NOVAS,MG,OK,[],[],"[['F0656/19', 'N', '11/04/2019 18:25', 'AD CLS...",{'response_status_code': 200}
2402,2019-04-23T05:57:20.724324-03:00,SNSI,MG0076,Santa Maria do Suaçuí,Público,SANTA MARIA DO SUAÇUÍ,MG,OK,[],[],"[['F0696/19', 'R', 'F0483/19', '15/04/2019 19:...",{'response_status_code': 200}
2404,2019-04-23T05:57:27.024296-03:00,SNSO,MG0072,Serro,Público,SERRO,MG,OK,[],[],"[['F0657/19', 'N', '11/04/2019 18:31', 'AD CLS...",{'response_status_code': 200}
2568,2019-04-23T05:57:23.982317-03:00,SWDM,MT0015,DIAMANTINO,Público,DIAMANTINO,MT,OK,[],[],"[['G0291/19', 'R', 'G1415/18', '11/03/2019 20:...",{'response_status_code': 200}
2577,2019-04-23T05:58:09.117523-03:00,SWHG,GO0032,Santa Helena de Goiás,Público,SANTA HELENA DE GOIÁS,GO,OK,[],[],"[['F0016/19', 'N', '07/01/2019 18:05', 'AD CLS...",{'response_status_code': 200}
2590,2019-04-23T05:54:59.498331-03:00,SWLC,GO0009,GEN. LEITE DE CASTRO,Público,RIO VERDE,GO,OK,[],[],"[['F0655/19', 'N', '11/04/2019 18:21', 'AD CLS...",{'response_status_code': 200}
2598,2019-04-23T05:55:32.983373-03:00,SWNR,MT0019,NORTELÂNDIA,Público,NORTELÂNDIA,MT,OK,[],[],"[['G0289/19', 'R', 'G1416/18', '11/03/2019 19:...",{'response_status_code': 200}
2602,2019-04-23T05:55:53.260331-03:00,SWPG,MT0010,PORTO DOS GAÚCHOS,Público,PORTO DOS GAÚCHOS,MT,OK,[],[],"[['G0287/19', 'R', 'G1417/18', '11/03/2019 19:...",{'response_status_code': 200}
2605,2019-04-23T05:56:10.100331-03:00,SWPL,MT0027,POSTO LEONARDO VILAS BOAS,Público,CHAPADA DOS GUIMARÃES,MT,OK,[],[],"[['G0290/19', 'R', 'G1418/18', '11/03/2019 20:...",{'response_status_code': 200}


##### Exportar para excel

In [ ]:
#ad_clsd_pbzpa.to_excel('notam_ad_clsd_pbzpa.xlsx', index=False, na_rep='')